# Splitting dataset into train, test and valid subsets 

In [4]:
import pandas as pd
import numpy as np
import os
import shutil

In [ ]:
def split_dataset_and_files(
    metadata_path,
    audio_folder,
    output_folder,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15
):
    """
    Dzieli zbiór chunków (audio_30_s_chunks_metadata) na train/val/test
    w proporcjach 70-15-15 (domyślnie), tak aby para (ARTIST_ID, ALBUM_ID)
    pojawiała się tylko w jednym podzbiorze.
    
    Następnie kopiuje fizycznie pliki MP3 do odpowiednich folderów:
      output_folder/train/..., output_folder/val/..., output_folder/test/...
    zachowując podstruktury folderów wg relatywnej ścieżki w kolumnie PATH.
    """
    # 1. Wczytaj dane
    metadata = pd.read_csv(metadata_path)

    # Sprawdzenie, czy sumują się do 1.0
    if not np.isclose(train_ratio + val_ratio + test_ratio, 1.0):
        raise ValueError("Proporcje zbiorów (train_ratio, val_ratio, test_ratio) muszą sumować się do 1.0.")

    # Upewnij się, że mamy kolumny ARTIST_ID i ALBUM_ID
    # W twoim projekcie mogą nazywać się inaczej, np. 'artist_id', 'album_id'
    # Dostosuj w razie potrzeby.
    required_cols = ["ARTIST_ID", "ALBUM_ID", "PATH"]
    for col in required_cols:
        if col not in metadata.columns:
            raise ValueError(f"Brak wymaganej kolumny '{col}' w pliku {metadata_path}.")

    # 2. Grupuj dane po (ARTIST_ID, ALBUM_ID)
    group_obj = metadata.groupby(["ARTIST_ID", "ALBUM_ID"])
    group_keys = list(group_obj.groups.keys())  # lista par (artist_id, album_id)
    
    # 3. Losowy podział albumów na zbiory
    np.random.shuffle(group_keys)  # in-place shuffle
    n = len(group_keys)
    train_end = int(n * train_ratio)
    val_end = train_end + int(n * val_ratio)
    
    train_keys = set(group_keys[:train_end])
    val_keys   = set(group_keys[train_end:val_end])
    test_keys  = set(group_keys[val_end:])

    # 4. Twórz podzbiory
    def assign_subset(row):
        key = (row["ARTIST_ID"], row["ALBUM_ID"])
        if key in train_keys:
            return "train"
        elif key in val_keys:
            return "val"
        elif key in test_keys:
            return "test"
        else:
            return "unknown"

    metadata["subset"] = metadata.apply(assign_subset, axis=1)
    
    train_data = metadata[metadata["subset"] == "train"]
    val_data   = metadata[metadata["subset"] == "val"]
    test_data  = metadata[metadata["subset"] == "test"]

    assert len(train_data) + len(val_data) + len(test_data) == len(metadata), \
        "Suma (train + val + test) nie zgadza się z całkowitą liczbą chunków"

    # 5. Twórz foldery wyjściowe
    os.makedirs(os.path.join(output_folder, "train"), exist_ok=True)
    os.makedirs(os.path.join(output_folder, "val"), exist_ok=True)
    os.makedirs(os.path.join(output_folder, "test"), exist_ok=True)

    # 6. Funkcja do kopiowania, zachowująca podfoldery
    def copy_files(df_subset, subset_name):
        subset_folder = os.path.join(output_folder, subset_name)
        for _, row in df_subset.iterrows():
            # Jeżeli PATH to np. '57/1294657_chunk_0.mp3'
            # To łączymy:
            src = os.path.join(audio_folder, row["PATH"])
            
            dest = os.path.join(subset_folder, row["PATH"])
            os.makedirs(os.path.dirname(dest), exist_ok=True)

            if os.path.exists(src):
                shutil.copy2(src, dest)
            else:
                print(f"[WARNING] Plik nie istnieje: {src}")

    print("Kopiowanie plików do 'train'...")
    copy_files(train_data, "train")
    print("Kopiowanie plików do 'val'...")
    copy_files(val_data, "val")
    print("Kopiowanie plików do 'test'...")
    copy_files(test_data, "test")

    return train_data, val_data, test_data

In [6]:
metadata_path = "../../datasets/jamendo/metadata/audio_30_s_chunks_metadata.csv"
audio_folder  = "../../datasets/jamendo/audio_30_s_chunks/"
output_folder = "../../datasets/jamendo/split_audio_dataset/"

# 70%, 15%, 15%
train_data, val_data, test_data = split_dataset_and_files(
    metadata_path,
    audio_folder,
    output_folder,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15
)

# Zapis do plików CSV
train_data.to_csv("../../datasets/jamendo/metadata/train_metadata.csv", index=False)
val_data.to_csv("../../datasets/jamendo/metadata/val_metadata.csv", index=False)
test_data.to_csv("../../datasets/jamendo/metadata/test_metadata.csv", index=False)

print("Zbiory danych i pliki audio zostały podzielone i zapisane.")
print(f"- Zbiór treningowy: {len(train_data)} przykładów")
print(f"- Zbiór walidacyjny: {len(val_data)} przykładów")
print(f"- Zbiór testowy: {len(test_data)} przykładów")

Kopiowanie plików do 'train'...
[WARNING] Plik nie istnieje: ../../datasets/jamendo/audio_30_s_chunks/../../datasets/jamendo/audio_30_s_chunks/57\1294657_chunk_0.mp3
[WARNING] Plik nie istnieje: ../../datasets/jamendo/audio_30_s_chunks/../../datasets/jamendo/audio_30_s_chunks/57\1294657_chunk_1.mp3
[WARNING] Plik nie istnieje: ../../datasets/jamendo/audio_30_s_chunks/../../datasets/jamendo/audio_30_s_chunks/57\1294657_chunk_2.mp3
[WARNING] Plik nie istnieje: ../../datasets/jamendo/audio_30_s_chunks/../../datasets/jamendo/audio_30_s_chunks/57\1294657_chunk_3.mp3
[WARNING] Plik nie istnieje: ../../datasets/jamendo/audio_30_s_chunks/../../datasets/jamendo/audio_30_s_chunks/57\1294657_chunk_4.mp3
[WARNING] Plik nie istnieje: ../../datasets/jamendo/audio_30_s_chunks/../../datasets/jamendo/audio_30_s_chunks/57\1294657_chunk_5.mp3
[WARNING] Plik nie istnieje: ../../datasets/jamendo/audio_30_s_chunks/../../datasets/jamendo/audio_30_s_chunks/75\31675_chunk_0.mp3
[WARNING] Plik nie istnieje: ../

KeyboardInterrupt: 